In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import log_loss
from tensorflow.keras.applications import EfficientNetB0

# Пути к данным
train_dir = "data/train_sample"
val_dir = "data/test_sample"

# Получаем список изображений
train_images = [os.path.join(train_dir, img) for img in os.listdir(train_dir) if img.endswith(('.jpg', '.png'))]
val_images = [os.path.join(val_dir, img) for img in os.listdir(val_dir) if img.endswith(('.jpg', '.png'))]

# Проверяем наличие изображений
assert len(train_images) > 0, "Ошибка: нет изображений в train_sample"
assert len(val_images) > 0, "Ошибка: нет изображений в test_sample"

# Функция загрузки изображений
def load_image(path):
    img = cv2.imread(path)
    if img is None:
        print(f"Предупреждение: изображение {path} не загружено")
        return None
    img = cv2.resize(img, (256, 256)) / 255.0
    return img

# Генератор загрузки обучающих данных
def data_generator(files, batch_size=32):
    batch_size = min(batch_size, len(files))
    while True:
        np.random.shuffle(files)
        for i in range(0, len(files), batch_size):
            batch_files = files[i:i + batch_size]
            images = [load_image(path) for path in batch_files if load_image(path) is not None]
            labels = [1 if "dog" in path.lower() else 0 for path in batch_files]
            yield np.array(images), np.array(labels)

# Создание модели
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = True

# Замораживаем первые 5 слоев
for layer in base_model.layers[:5]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='swish'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='swish'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer=AdamW(learning_rate=0.00001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Обучение модели
history = model.fit(
    data_generator(train_images, batch_size=32),
    steps_per_epoch=len(train_images) // 32,
    epochs=30,
    validation_data=data_generator(val_images, batch_size=32),
    validation_steps=len(val_images) // 32
)

# Визуализация обучения
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Точность модели')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Потери модели')

plt.show()

# Сохранение модели
model.save('cats_vs_dogs_efficientnet.keras')

# Проверка Log Loss
test_images, test_labels = next(data_generator(val_images, batch_size=len(val_images)))
y_pred = model.predict(test_images)
logloss = log_loss(test_labels, y_pred)
print(f"Log Loss модели: {logloss:.4f}")

# Функция предсказания
def predict_image(img_path):
    img = load_image(img_path)
    if img is None:
        return "Файл не существует ❌"
    img_array = np.expand_dims(img, axis=0)
    prediction = model.predict(img_array)
    return "Собака 🐶" if prediction[0] > 0.5 else "Кошка 🐱"

# Пример использования
img_path = "data/test_sample/2285.jpg"
print(predict_image(img_path))

d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 31s/step - accuracy: 0.5750 - loss: 0.1364 - val_accuracy: 0.0000e+00 - val_loss: 0.1288
Epoch 2/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 80s 36s/step - accuracy: 0.7510 - loss: 0.0781 - val_accuracy: 0.8000 - val_loss: 0.1172
Epoch 3/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 92s 26s/step - accuracy: 0.8484 - loss: 0.0446 - val_accuracy: 1.0000 - val_loss: 0.0937
Epoch 4/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 74s 26s/step - accuracy: 0.8995 - loss: 0.0380 - val_accuracy: 1.0000 - val_loss: 0.0854
Epoch 5/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 64s 15s/step - accuracy: 0.9182 - loss: 0.0259 - val_accuracy: 1.0000 - val_loss: 0.0833
Epoch 6/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 18s/step - accuracy: 0.9359 - loss: 0.0234 - val_accuracy: 1.0000 - val_loss: 0.0866
Epoch 7/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 58s 18s/step - accuracy: 0.9937 - loss: 0.0090 - val_accuracy: 1.0000 - val_loss: 0.0989
Epoch 8/75
3/3 ━━━━━━━━━━━━━━━━━━━━ 72s 19s/step - accuracy: 0.9469 - loss: 0.0172 - val_accuracy: 0.3000 - val_loss: 0.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Загрузка изображения
img_path = val_images[6]  # Используем первое изображение из валидационного набора
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Масштабирование пикселей

# Предсказание
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Это собака 🐶")
else:
    print("Это кошка 🐱")

InvalidArgumentError: Graph execution error:

Detected at node sequential_11_1/flatten_11_1/Reshape defined at (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code

  File "d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  Fil

In [ ]:
# Сохранение модели
model.save('cats_vs_dogs_model.h5')

# Загрузка модели
from tensorflow.keras.models import load_model
loaded_model = load_model('cats_vs_dogs_model.h5')

# Проверим, успешно ли загружена модель
print("Модель загружена успешно!")

Модель загружена успешно!


In [ ]:
from sklearn.metrics import log_loss

# Предсказания модели на тестовом наборе
y_pred = model.predict(X_test)

# Преобразуем предсказания в вероятности (если модель дает значения от 0 до 1, они уже в нужном формате)
logloss = log_loss(y_test, y_pred)

print(f"Log Loss модели: {logloss:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Log Loss модели: 0.7047
